#  - IMPORTS - 

In [55]:
import numpy as np 
import matplotlib.pyplot as plt
import os
from keras import layers
from keras.models import Model
from tensorflow.keras.models import Model
from keras.models import load_model
from keras import callbacks
import cv2
import string
import time

#  - INIT - 

In [56]:
imgshape=(50,200,1)

# n is the total number of images in the training dataset
n=len(os.listdir("./captcha_dataset/train_set"))
n

# n char is the total nu.of possible characters
character = string.ascii_lowercase + "0123456789"
nchar = len(character)
nchar

36

#  - PREPROCESS - 

In [57]:
def preprocess():
    x = np.zeros((n,50,200,1)) # 1030*5*200*1
    y = np.zeros((5,n,nchar)) # 5*1030*36

    # enumerate() gives us [(0,filename),(1,filename),(2,filename)]
    # i represents the index
    # pic contains the filename of the image

    for i, pic in enumerate(os.listdir("./captcha_dataset/train_set/")):
        img = cv2.imread(os.path.join("./captcha_dataset/train_set/", pic), cv2.IMREAD_GRAYSCALE)
        pic_target = pic[:-4] #This is the target we are trying to achieve 2b827 for example after dropping the file extension

        if len(pic_target) < 6: # Making sure captcha is less than 5 letters long
            img = img/255.0
            img = np.reshape(img, (50, 200, 1))

            target=np.zeros((5,nchar))

            for j, k in enumerate(pic_target):
                #j iterates from 0 to 4(5 letters in captcha)
                #k denotes the letter in captcha which is to be scanned
                index = character.find(k) #index stores the position of letter k of captcha in the character string
                target[j, index] = 1 #replaces 1 in the target array at the position of the letter in captcha

            x[i] = img
            y[:,i] = target
            
    return x,y

#  - Model Creation - 3 Convolutional Layers + 3 Max Pooling Layers + Others - 

In [ ]:
def createmodel():
    img = layers.Input(shape=imgshape)
    conv1 = layers.Conv2D(32,(3,3), padding='same', activation='relu',name='conv1')(img)
    mp1 = layers.MaxPooling2D((2,2),padding='same')(conv1)
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D((2,2),padding='same')(conv2)
    conv3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp2)
    bn = layers.BatchNormalization()(conv3)
    mp3 = layers.MaxPooling2D((2,2),padding='same')(bn)
    flat = layers.Flatten()(mp3)

    outs = []
    for _ in range(5): #for 5 letters of captcha
        dens1 = layers.Dense(64, activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1) #drops 0.5 of nodes
        res = layers.Dense(nchar, activation='sigmoid')(drop)

        outs.append(res) #result of layers
    model = Model(img, outs) #create model
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy","accuracy","accuracy","accuracy","accuracy"])
    return model

model = createmodel()
model.summary()


# - Model Training With 30 Epochs -

In [ ]:
x,y = preprocess()
hist = model.fit(x, [y[0], y[1], y[2], y[3], y[4]], batch_size=32, epochs=30, validation_split=0.2)


# - Loss On Trianing Set -

In [ ]:
preds = model.evaluate(x, [y[0], y[1], y[2], y[3], y[4]])
print ("Loss on training set= " + str(preds[0]))

# - Loss On Testing Set - (Unseen Data)

In [ ]:
# Preprocess Test Data the same way we did for Training Data
def preprocess():
    x = np.zeros((40,50,200,1)) # 1030*5*200*1
    y = np.zeros((5,40,nchar)) # 5*1030*36

    # enumerate() gives us [(0,filename),(1,filename),(2,filename)]
    # i represents the index
    # pic contains the filename of the image

    for i, pic in enumerate(os.listdir("./captcha_dataset/test_set/")):
        img = cv2.imread(os.path.join("./captcha_dataset/test_set/", pic), cv2.IMREAD_GRAYSCALE)
        pic_target = pic[:-4] #This is the target we are trying to achieve 2b827 for example after dropping the file extension

        if len(pic_target) < 6: # Making sure captcha is less than 5 letters long
            img = img/255.0
            img = np.reshape(img, (50, 200, 1))

            target=np.zeros((5,nchar))

            for j, k in enumerate(pic_target):
                #j iterates from 0 to 4(5 letters in captcha)
                #k denotes the letter in captcha which is to be scanned
                index = character.find(k) #index stores the position of letter k of captcha in the character string
                target[j, index] = 1 #replaces 0 with 1 in the target array at the position of the letter in captcha

            x[i] = img
            y[:,i] = target
            
    return x,y

x,y = preprocess()

preds = model.evaluate(x, [y[0], y[1], y[2], y[3], y[4]])
print ("Loss on testing set= " + str(preds[0]))

# - Predict Function - 

In [ ]:
def predict(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

    if img is not None: #image foud at file path
        img = img / 255.0 #Scale image
    else:
        print("Not detected")

    res = np.array(model.predict(img[np.newaxis, :, :, np.newaxis]))
    result = np.reshape(res, (5, 36))
    k_ind = []
    probs = []
    for i in result:
        k_ind.append(np.argmax(i))

    capt = ''
    for k in k_ind:
        capt += character[k]
    return capt 

# - Model Predicting Unseen Data -

In [ ]:
for i, pic in enumerate(os.listdir("./captcha_dataset/test_set/")):
    img = cv2.imread(f'./captcha_dataset/test_set/{pic}',cv2.IMREAD_GRAYSCALE)
    plt.imshow(img, cmap='gray') 
    plt.show()
    print("Predicted Captcha =",predict(f'./captcha_dataset/test_set/{pic}'))
    plt.clf()
    time.sleep(5)